In [1]:
import pandas as pd
import os
from glob import glob
import numpy as np
import json
from itertools import izip

In [2]:
def get_env_var(handle):
    ''' Get an environment variable given the handle for the bash variable'''
    tmp = os.getenv(handle)
    if not tmp:
        raise LookupError("Environment variable: {} not set.".format(handle))
    return tmp.strip("'")

In [3]:
def format_md_img(link_name, rel_plot_loc):
    img_formatter="[[{}/{}]]".format(link_name, rel_plot_loc)
    return img_formatter

In [4]:
def print_table(list_of_headers):
    fmter =  "| {} {}".format("{}", " {} ")
    for header in list_of_headers:
        fmter = fmter.format("{} {}".format(header, "| {}"), "{}")
    fmter = fmter.format(" ", " ")
    return fmter

In [5]:
class  Experiment(dict):
    def __init__(self, name):
        self.name = name
        self.folds = []
        self.converged_epochs = []
        self.trained_paths = []
      
    def __repr__(self):
        return str(vars(self))
    
    def __str__(self):
        return json.dumps(vars(self), indent=2)
        
    def set_convereged_epoch(self, epoch, train_path, fold=None):        
        self.folds.append(fold)
        self.converged_epochs.append(epoch)
        self.trained_paths.append(train_path)
        

In [6]:
home=get_env_var("HOME")
base="{}/labgits/lab-notebook-caceres.wiki/images/20180815_Paper_Retrains/".format(home)

In [7]:
home_save_dir = get_env_var("HOME_SAVE_BASE")
srv_save_dir = get_env_var("DATA_SAVE_BASE")
proj_dir = get_env_var("NMEW_PROJ_BASE")

sneg_pnrs=[1.0, 1.2222, 0.8182, 1.5, 19.0, 9.0, 4.0, 2.3333, 0.6666, 0.4286, 0.25, 0.1111, 0.0753]
sneg_pnrs = sorted(sneg_pnrs)
fold_tmplt = "fold_[0-9]*/pnr_*/*.png"

# github formatting info:
github_wiki_link="https://github.com/keiserlab/lab-notebook-caceres/wiki/images"
expt_name = "20180815_Paper_Retrains"
github_wiki_expt = "{}/{}".format(github_wiki_link, expt_name)

neg_rm_dir = "{}/all_negs_stochastic/".format(base)

metrics_for_convergance = ['matthews-corrcoef_binary-5.0_test', 'matthews-corrcoef_binary-6.0_test', 'r2_test']
experiments = []

parent_dir = os.path.join(neg_rm_dir, "NEG_RM_RATIOS")
png_files = glob(os.path.join(parent_dir, fold_tmplt))

In [8]:
experiments = []
converged_epochs = ("{}/output/20180815_Paper_Retrains/predictions/experiments.json".format(home_save_dir))
with open(converged_epochs, "r") as fp:
    expts = json.load(fp)

expt_list = []
for e in expts:
    tmp = Experiment(e["name"])
    for epoch, path, fold in izip(e["converged_epochs"], e["trained_paths"], e["folds"]):
        tmp.set_convereged_epoch(epoch, path, fold)
    experiments.append(tmp)
    del(tmp)    
    
expt_dict = {float(i.name.split("_")[-1]): dict(zip([int(j.split("_")[1]) for j in i.folds], i.converged_epochs)) for i in experiments}

In [9]:
expt_dict

{0.0753: {0: 321, 1: 256, 2: 386, 3: 216, 4: 492},
 0.1111: {0: 476, 1: 176, 2: 456, 3: 499, 4: 475},
 0.25: {0: 272, 1: 100, 2: 468, 3: 326, 4: 231},
 0.4286: {0: 163, 1: 100, 2: 417, 3: 162, 4: 454},
 0.6666: {0: 240, 1: 200, 2: 0, 3: 451, 4: 455},
 0.8182: {0: 499, 1: 200, 2: 81, 3: 388, 4: 24},
 1.0: {0: 209, 1: 420, 2: 322, 3: 454, 4: 330},
 1.2222: {0: 200, 1: 133, 2: 402, 3: 200, 4: 359},
 1.5: {0: 300, 1: 100, 2: 303, 3: 0, 4: 0},
 2.3333: {0: 499, 1: 300, 2: 97, 3: 242, 4: 99},
 4.0: {0: 339, 1: 67, 2: 105, 3: 471, 4: 0},
 9.0: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0},
 19.0: {0: 0, 1: 0, 2: 0, 3: 0, 4: 121}}

In [10]:
img_order = ['train_loss','test_loss', 'test_no_gt_loss', 'test_sneg_loss']
for ratio in sneg_pnrs:
    png_subset = [f for f in png_files if str(ratio) in f]
    headers = ["PNR_{}".format(ratio)]
    headers.extend(img_order)
    print(print_table(headers))
    print(print_table([":---"]*len(headers)))
    for f in np.arange(0, 5, 1):
        name_fmter = "fold_{} best epoch: {}".format(str(f), expt_dict[ratio][f])
        fold_fmter = "fold_{}".format(str(f))
        fold_pngs = [i for i in png_subset if fold_fmter in i]
        fold_pngs = sorted(fold_pngs, key=lambda x: img_order.index(x.split("/")[-1].split(".")[0]))
        md_print_fmt = [name_fmter] + [format_md_img(github_wiki_expt, i.split("//")[-1]) for i in fold_pngs]
        print(print_table(md_print_fmt))
    print("\n\n")

| PNR_0.0753 | train_loss | test_loss | test_no_gt_loss | test_sneg_loss |      
| :--- | :--- | :--- | :--- | :--- |      
| fold_0 best epoch: 321 | [[https://github.com/keiserlab/lab-notebook-caceres/wiki/images/20180815_Paper_Retrains/all_negs_stochastic/NEG_RM_RATIOS/fold_0/pnr_0.0753/train_loss.png]] | [[https://github.com/keiserlab/lab-notebook-caceres/wiki/images/20180815_Paper_Retrains/all_negs_stochastic/NEG_RM_RATIOS/fold_0/pnr_0.0753/test_loss.png]] | [[https://github.com/keiserlab/lab-notebook-caceres/wiki/images/20180815_Paper_Retrains/all_negs_stochastic/NEG_RM_RATIOS/fold_0/pnr_0.0753/test_no_gt_loss.png]] | [[https://github.com/keiserlab/lab-notebook-caceres/wiki/images/20180815_Paper_Retrains/all_negs_stochastic/NEG_RM_RATIOS/fold_0/pnr_0.0753/test_sneg_loss.png]] |      
| fold_1 best epoch: 256 | [[https://github.com/keiserlab/lab-notebook-caceres/wiki/images/20180815_Paper_Retrains/all_negs_stochastic/NEG_RM_RATIOS/fold_1/pnr_0.0753/train_loss.png]] | [[https://git